In [1]:
import os
import csv
from countries_languages import country_to_language

import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime

from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


def remove_accents(text):
    return unidecode(text)

def get_country_code(country):
    matching_rows = countries_codes.loc[countries_codes['Country'] == country, ' Code']
    return matching_rows.iloc[0] if not matching_rows.empty else None

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

#PLAYER DATA - MARKET VALUE AND SALARY
leagues_salary = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1//CSVs we use often/latest_capology_data_money_fixed.csv')

leagues_value = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/MarketValuesComplete.csv')


leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')

#COUNTRY CODES
countries_codes = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/countries_and_codes.csv')

#FIFA RANKINGS
fifa_rankings = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/fifa_ranking-2023-10-26 - fifa_ranking-2023-07-20.csv')

#MATCH DATA - LINEUPS AND RESULTS
# lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_march_24.csv')
lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_april_29.csv')
results_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_results_data_march_24.csv')


leagues_value['Accents Removed'] = leagues_value['Name'].apply(lambda x: unidecode(x))
leagues_value['Team 1 Code'] = leagues_value['Nationality'].apply(get_country_code)

countries_codes['Accents Removed'] = countries_codes['Country'].apply(remove_accents)




/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/AssemblyHelpers.py:54: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)
/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_31625/899278855.py:63: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')


In [2]:
#HELPER METHODS
##SELENIUM WRAPPER FUNCTION 

def useSeleniumToFindMoney(
        NAME_TO_FIND, #the name it is looking up
        NATIONALITY_INPUT_PLAYER, #natl_test
        DATE_INPUT_MATCH, #input_year_test
        SALARY_BOOLEAN):
    if(SALARY_BOOLEAN == False):
        print('finding salary with selenium')
        marketvalue = AssemblyHelpers.seleniumLookUpValueWrapperFunction(NAME_TO_FIND, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH)
        return marketvalue
    elif(SALARY_BOOLEAN == True):
        salary = AssemblyHelpers.capology_selenium_lookup(NAME_TO_FIND, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH)
        return salary

    else:
        return 0



#MATCH DATE FILTER 
def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''



#FINDING MONEY INFO USSING PLAYER NAME
def tryToFindPlayerMoney(
        DATABASE_RETURN_MESSAGE, #what the DB said when trying to find money info
        NAME_TO_FIND, #the name it found in the DB (name_match[0] originally)
        NATIONALITY_INPUT_PLAYER, #natl_test
        DATE_INPUT_MATCH, #input_year_test
        SALARY_BOOLEAN, #IF TRUE, SALARIES. OTHERWISE VALUES
        ORIGINAL_SEARCH_NAME #name_match[2]
        ):

    if((DATABASE_RETURN_MESSAGE == "not in DB any of 3 seasons.") or (DATABASE_RETURN_MESSAGE == "asswipe was in DB in before or after season")):
        print('newwwnewnenwenewnwenwenwew', DATABASE_RETURN_MESSAGE)
        
        if DATABASE_RETURN_MESSAGE == 'asswipe was in DB in before or after season':
            money_thisplayer = [0, 'Was in  DB before or after season', True, True]
            return money_thisplayer

        
        
        

        INTERNET_LOOKUP_RESULTING_NAMES = AssemblyHelpers.false_name_match_lookup(ORIGINAL_SEARCH_NAME, NATIONALITY_INPUT_PLAYER)
        NUM_NAMES_REMAINING = len(list(INTERNET_LOOKUP_RESULTING_NAMES))
        print(INTERNET_LOOKUP_RESULTING_NAMES)
        #1 MATCH
        if(NUM_NAMES_REMAINING == 1):
            #FIND INFO USING SELENIUM
            # NAME_FOR_SELENIUM_SEARCH = next(iter(INTERNET_LOOKUP_RESULTING_NAMES))
            # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH, SALARY_BOOLEAN)
            # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
            money_thisplayer = [0, 'Lookup Required', True, False]
            return money_thisplayer
        #MULTIPLE MATCHES
        elif(NUM_NAMES_REMAINING >= 2):
            print('goes to FUDOM')
            #FILTER THE NAMES USING THE DATE OF THE MATCH
            list_to_filter_with_match_date = list(INTERNET_LOOKUP_RESULTING_NAMES)
            #NEW FUNCTION 3 
            money_thisplayer, name_results = filter_using_date_of_match(list_to_filter_with_match_date, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH, SALARY_BOOLEAN) 
            return money_thisplayer
        #0 MATCHES
        elif(NUM_NAMES_REMAINING == 0):
            #This is return case 9
            money_thisplayer = [0, 'Online Lookup Required', True, False] 
            return money_thisplayer
    
    else:
        money_thisplayer = [0, 'Online Lookup Required', True, False]
        return money_thisplayer
    


    #old
    # elif(DATABASE_RETURN_MESSAGE == "was in DB in before or after season"):
    #     0==0
    

##OVERARCHING Controller FUNCTION 
def new_overarching(starter_number, name_input, nationality_input, date_input, salary_boolean, competition_input, combination_dict_matches, combination_dict_NON_matches, combination_dict_multi_matches):
    """
    Inputs: Name, Date, Nationality, Salary Boolean 
        salary_boolean
            true – salary
            false – Market Value

    Outputs: Money_thisplayer, name_match
    Outputs to add: error, error type, error msg
    """

    #SETTING INITIAL VALUES
    example_problem = name_input
    natl_test = nationality_input
    input_year_test = date_input

    if 'U-20' in natl_test:
        print('u-20 team')
        natl_test = natl_test.split(' U-20')[0]

    corrrect_code = countries_codes[countries_codes['Country'] == natl_test][' Code'].unique()[0]
    

    if salary_boolean == True:
        0==0
        #dataset_nationality = leagues_salary[leagues_salary['Team 1 Code'] == corrrect_code]['Name'].unique()
    else:
        dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == corrrect_code]['Name'].unique()

    money_thisplayer = [0, 'not done yet', True, False]
    secondmatch_required = False

    #FIRST TRY LOOKING FOR NAME
    name_match = AssemblyHelpers.lookup_name(example_problem, natl_test, input_year_test, salary_boolean, competition_input)
    
    match_case = name_match[1]
    original_db_name_for_search = name_match[2]
    print('match is', name_match)

    

    if(match_case == 'single'):
        NAME_MATCH_FIRST_TRY = name_match[0]
        #print(NAME_MATCH_FIRST_TRY)
        money_thisplayer = AssemblyHelpers.find_money_info_from_name(NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values)

        money_found = money_thisplayer[0]
        search_db_return_case = money_thisplayer[1]
        online_lookup_required = money_thisplayer[2]

        if ((money_found == 0) & (online_lookup_required == True)):         #call the find_money_info_extended once you write it. inputs are = (NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, name_match[2])
            #NEW FUNCTION 1
            if '.' in input_year_test:
                yearstr = input_year_test.split(".")[2]
                full_num = int('20' + yearstr)
            elif '-' in input_year_test:
                if len(input_year_test.split("-")[2]) == 4:
                    yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                    full_num = int(yearstr)
                elif len(input_year_test.split("-")[2]) == 2:
                    if len(input_year_test.split("-")[0]) == 4:
                        yearstr = input_year_test.split("-")[0]
                        full_num = int(yearstr)
                    else:
                        yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                        full_num = int('20' + yearstr)
                else:
                    print('broke on year parsing', input_year_test.split("-"))
                # elif len(input_year_test.split("-")[0]) == 4:
                    # yearstr = input_year_test.split("-")[0]
                    # full_num = yearstr
            
            #print(f'singi trying {NAME_MATCH_FIRST_TRY} and {int('20' + yearstr)}')
            player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num)].reset_index()
            
            if len(player_bigvalue_dataset) >= 1:
                #print(player_bigvalue_dataset.at[0, 'Market Value'])
                #money_thisplayer[0] =
                #print('working youre good')
                money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                        #money_thisplayer[2] = True
                        money_thisplayer[1] = 'Large Dataset had 0'
            else:
                if len(leagues_value_large[((leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num - 1)) | 
                                       ((leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num+ 1))]) >= 2:
                    money_thisplayer = [0, 'Not in either dataset for currect season but present in both seasons around match', False, True]
                    # leagues_value_large[((leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num - 1)) | 
                    #                    ((leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num + 1))]['Market Value'].mean()
                else:
                    if ((leagues_value[leagues_value['Name'] == NAME_MATCH_FIRST_TRY]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY]['Season'].max() == (full_num - 1))):
                        money_last_season = leagues_value[(leagues_value['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                        money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                    else:
                        money_thisplayer = tryToFindPlayerMoney(search_db_return_case, NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
            
        elif(search_db_return_case == 'working'):
            0==0
            #return to the end basically
        elif(search_db_return_case == 'working - imputed'):
            0==0
            #when ur done this should be removed. go into the function find_money_info and remove case
                    
    elif(match_case == 'multiple'):
        print('multi match')

        if '.' in input_year_test:
            yearstr = input_year_test.split(".")[2]
            full_num = int('20' + yearstr)
        elif '-' in input_year_test:
            if len(input_year_test.split("-")[2]) == 4:
                yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                full_num = int(yearstr)
            elif len(input_year_test.split("-")[2]) == 2:
                if len(input_year_test.split("-")[0]) == 4:
                    yearstr = input_year_test.split("-")[0]
                    full_num = int(yearstr)
                else:
                    yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                    full_num = int('20' + yearstr)

        combination_key = (example_problem, full_num)

        if combination_key in combination_dict_matches:

            print('combo is in matches dict')
            #correct_name = combination_dict_matches[combination_key]
            #print('correct name is ', correct_name)

            SINGLE_NAME_MATCH_CORRECT_SEASON = combination_dict_matches[combination_key]
                
            money_thisplayer = AssemblyHelpers.find_money_info_from_name(SINGLE_NAME_MATCH_CORRECT_SEASON[0], natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 

            money_found = money_thisplayer[0]
            search_db_return_case = money_thisplayer[1]
            online_lookup_required = money_thisplayer[2]

            if ((money_found == 0) & (online_lookup_required == True)):  
                #NEW FUNCTION 1     
                if '.' in input_year_test:
                    yearstr = input_year_test.split(".")[2]
                    full_num = int('20' + yearstr)
                elif '-' in input_year_test:
                    if len(input_year_test.split("-")[2]) == 4:
                        yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                        full_num = int(yearstr)
                    elif len(input_year_test.split("-")[2]) == 2:
                        if len(input_year_test.split("-")[0]) == 4:
                            yearstr = input_year_test.split("-")[0]
                            full_num = int(yearstr)
                        else:
                            yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                            full_num = int('20' + yearstr)
                    else:
                        print('broke on year parsing', input_year_test.split("-"))
        
                #print(f'multiii trying {SINGLE_NAME_MATCH_CORRECT_SEASON} and {int('20' + yearstr)}')
                if type(SINGLE_NAME_MATCH_CORRECT_SEASON) == list:
                    SINGLE_NAME_MATCH_CORRECT_SEASON = SINGLE_NAME_MATCH_CORRECT_SEASON[0]
                player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value_large['Season'] == full_num)].reset_index()
                
                if len(player_bigvalue_dataset) >= 1:
                    #print(player_bigvalue_dataset.at[0, 'Market Value'])
                    #money_thisplayer[0] =
                    #print('working youre good')
                    money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                    if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                        #money_thisplayer[2] = True
                        money_thisplayer[1] = 'Large Dataset said 0'
                else:
                    if ((leagues_value[leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1))):
                        money_last_season = leagues_value[(leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                        money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                    else:
                        money_thisplayer = tryToFindPlayerMoney(search_db_return_case, SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                        
            elif(search_db_return_case == 'working'):
                0==0
                #return to the end basically
            elif(search_db_return_case == 'working - imputed'):
                0==0
                #when ur done this should be removed. go into the function find_money_info and remove case
            
            #in this situation, you need to do find money info from name. 
            #use the name that is in the dict 
        elif combination_key in combination_dict_NON_matches:
            print('combination in non-matches dict')#if you find it in the non matches dictionary
            #save the list of names it returned 
            #then return money thisplayer 
            NAMES_MATCHES_LIST_CORRECT_SEASON = combination_dict_NON_matches[combination_key]
            money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, ORIGINAL NAME WAS {example_problem}', True, False]
        elif combination_key in combination_dict_multi_matches:
            #stop the loop here 
            filtered_names_match_date = combination_dict_multi_matches[combination_key]
            money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {filtered_names_match_date}', True, False]
        else:
            print('combo is not in matches, multi-matches or non-matches dict')
        
            NAMES_MATCHES_LIST = name_match[0]
            
            
            #FILTER THE NAMES USING THE YEAR OF THE MATCH
            NAMES_MATCHES_LIST_CORRECT_SEASON = AssemblyHelpers.filter_candidates_using_year(starter_number, NAMES_MATCHES_LIST, natl_test, input_year_test, salary_boolean)
            

            NUMBER_OF_NAMES_MATCHING = len(NAMES_MATCHES_LIST_CORRECT_SEASON)
            #print(f'left is {NUMBER_OF_NAMES_MATCHING}')
            
            if(NUMBER_OF_NAMES_MATCHING == 1): #YELLOW 2
                
                
                SINGLE_NAME_MATCH_CORRECT_SEASON = NAMES_MATCHES_LIST_CORRECT_SEASON[0]
                
                money_thisplayer = AssemblyHelpers.find_money_info_from_name(SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 

                money_found = money_thisplayer[0]
                search_db_return_case = money_thisplayer[1]
                online_lookup_required = money_thisplayer[2]

                if ((money_found == 0) & (online_lookup_required == True)):  
                    #NEW FUNCTION 1     
                    if '.' in input_year_test:
                        yearstr = input_year_test.split(".")[2]
                        full_num = int('20' + yearstr)
                    elif '-' in input_year_test:
                        if len(input_year_test.split("-")[2]) == 4:
                            yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                            full_num = int(yearstr)
                        elif len(input_year_test.split("-")[2]) == 2:
                            if len(input_year_test.split("-")[0]) == 4:
                                yearstr = input_year_test.split("-")[0]
                                full_num = int(yearstr)
                            else:
                                yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                                full_num = int('20' + yearstr)
                        else:
                            print('broke on year parsing', input_year_test.split("-"))
            
                    #print(f'multiii trying {SINGLE_NAME_MATCH_CORRECT_SEASON} and {int('20' + yearstr)}')
                    print(full_num, SINGLE_NAME_MATCH_CORRECT_SEASON)
                    player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value_large['Season'] == full_num)].reset_index()
                    
                    if len(player_bigvalue_dataset) >= 1:
                        #print(player_bigvalue_dataset.at[0, 'Market Value'])
                        #money_thisplayer[0] =
                        #print('working youre good')
                        money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                        if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                            #money_thisplayer[2] = True
                            money_thisplayer[1] = 'Large Dataset said 0'
                    else:
                        if ((leagues_value[leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1))):
                            money_last_season = leagues_value[(leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                            money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                        else:
                            money_thisplayer = tryToFindPlayerMoney(search_db_return_case, SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                            
                elif(search_db_return_case == 'working'):
                    0==0
                    #return to the end basically
                elif(search_db_return_case == 'working - imputed'):
                    0==0
                    #when ur done this should be removed. go into the function find_money_info and remove case
                
            elif(NUMBER_OF_NAMES_MATCHING >= 2):
                #use match date to filter 
                #print('list of names is ', NAMES_MATCHES_LIST_CORRECT_SEASON)
                #print('we are here', NUMBER_OF_NAMES_MATCHING)
                if('Olympics' in competition_input):
                    money_thisplayer = [0, F'Olympics row. Couldn\'t filter names using Transfermarkt. remaining matches were {NAMES_MATCHES_LIST_CORRECT_SEASON}', True, False]
                    #the filtering is not going to work 
                else:
                
                    filtered_names_match_date = AssemblyHelpers.multiNameMatchDateLookup(NAMES_MATCHES_LIST_CORRECT_SEASON, natl_test, input_year_test) 

                    #1 MATCH
                    if(type(filtered_names_match_date) == str): #YELLOW 3
                        
                        if '.' in input_year_test:
                            yearstr = input_year_test.split(".")[2]
                            full_num = int('20' + yearstr)
                        elif '-' in input_year_test:
                            if len(input_year_test.split("-")[2]) == 4:
                                yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                                full_num = int(yearstr)
                            elif len(input_year_test.split("-")[2]) == 2:
                                if len(input_year_test.split("-")[0]) == 4:
                                    yearstr = input_year_test.split("-")[0]
                                    full_num = int(yearstr)
                                else:
                                    yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                                    full_num = int('20' + yearstr)
                        #print('we here', filtered_names_match_date, full_num, example_problem)
                        combination_key = (example_problem, full_num)
                        combination_dict_matches[combination_key] = [filtered_names_match_date]
                        print(f'added key {combination_key} to dict. value is {filtered_names_match_date}')
                        #at this point, example problem and full_num become a combination 
                        #that combination should be a key, and the value should be filtered_names_match_date
                        #add a new key+value combination to the dictionary here. 

                        NAME_MATCH_SINGLE = filtered_names_match_date

                        #name_match[0] = NAME_MATCH_SINGLE
                        #print('we hereeeee', NAME_MATCH_SINGLE)
                        money_thisplayer = AssemblyHelpers.find_money_info_from_name(NAME_MATCH_SINGLE, natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 
                        
                        money_found = money_thisplayer[0]
                        search_db_return_case = money_thisplayer[1]
                        online_lookup_required = money_thisplayer[2]

                        if ((money_found == 0) & (online_lookup_required == True)):         #call the find_money_info_extended once you write it. inputs are = (NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, name_match[2])
                            # NAME_FOR_SELENIUM_SEARCH = NAME_MATCH_SINGLE
                            # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, natl_test, input_year_test, salary_boolean)
                            # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {salary_boolean}', True, False]

                            if len(leagues_value_large[(leagues_value_large['Name'] == NAME_MATCH_SINGLE) & (leagues_value_large['Season'] == full_num)]) >= 1:
                                money_thatseason_large_db = leagues_value_large[(leagues_value_large['Name'] == NAME_MATCH_SINGLE) & (leagues_value_large['Season'] == full_num)]['Market Value'].max()
                                money_thisplayer = [money_thatseason_large_db, 'Used Large Dataset', False, False]

                            elif ((leagues_value[leagues_value['Name'] == NAME_MATCH_SINGLE]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == NAME_MATCH_SINGLE]['Season'].max() == (full_num - 1))):
                                money_last_season = leagues_value[(leagues_value['Name'] == NAME_MATCH_SINGLE) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                                money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                            else:
                                #money_thisplayer = tryToFindPlayerMoney(search_db_return_case, NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                                money_thisplayer = [0, 'Lookup Required', True, False]
                            #return money_thisplayer
                            
                        elif(search_db_return_case == 'working'):
                            0==0
                            #return to the end basically
                        elif(search_db_return_case == 'working - imputed'):
                            0==0
                            #when ur done this should be removed. go into the function find_money_info and remove case


                    #MULTIPLE MATCHES
                    elif(len(filtered_names_match_date) >= 2): #RED 6

                        print('added a row to multi matches dictionary.')
                        combination_key_multi_match = (example_problem, full_num)
                        combination_dict_multi_matches[combination_key_multi_match] = [filtered_names_match_date]
                        
                        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {filtered_names_match_date}', True, False]
                        #NNNNEW - add to the dictionary here 
                        #return money_thisplayer, name_match
                    #0 MATCHES
                    elif(len(filtered_names_match_date) == 0): #RED 5
                        #print('we are here. bahrain case')
                        if '.' in input_year_test:
                            yearstr = input_year_test.split(".")[2]
                            full_num = int('20' + yearstr)
                        elif '-' in input_year_test:
                            if len(input_year_test.split("-")[2]) == 4:
                                yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                                full_num = int(yearstr)
                            elif len(input_year_test.split("-")[2]) == 2:
                                if len(input_year_test.split("-")[0]) == 4:
                                    yearstr = input_year_test.split("-")[0]
                                    full_num = int(yearstr)
                                else:
                                    yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                                    full_num = int('20' + yearstr)

                        combination_key_nomatch = (example_problem, full_num)
                        combination_dict_NON_matches[combination_key_nomatch] = [NAMES_MATCHES_LIST_CORRECT_SEASON]
                        print(f'added a row to non matches dict. player was {example_problem} in {full_num}.')
                        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, ORIGINAL NAME WAS {example_problem}', True, False]
                        #return money_thisplayer, name_match
                    
            elif(NUMBER_OF_NAMES_MATCHING == 0):#RETURN CASE 4

                print('here rc4')
                
                money_thisplayer, name_results = filter_using_date_of_match(NAMES_MATCHES_LIST, natl_test, input_year_test, salary_boolean) 
                if money_thisplayer == [0, 'Lookup Required', True, False]:
                    SINGLE_NAME_MATCH_CORRECT_SEASON = name_results
                    combination_key = (example_problem, full_num)
                    combination_dict_matches[combination_key] = [SINGLE_NAME_MATCH_CORRECT_SEASON]
                    print(f'added key {combination_key} to dict. value is {SINGLE_NAME_MATCH_CORRECT_SEASON}')
                    player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value_large['Season'] == full_num)].reset_index()
                    
                    if len(player_bigvalue_dataset) >= 1:
                        #print(player_bigvalue_dataset.at[0, 'Market Value'])
                        #money_thisplayer[0] =
                        #print('working youre good')
                        money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                        if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                            #money_thisplayer[2] = True
                            money_thisplayer[1] = 'Large Dataset said 0'
                    else:
                        if ((leagues_value[leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1))):
                            money_last_season = leagues_value[(leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                            money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                        else:
                            money_thisplayer = tryToFindPlayerMoney(money_thisplayer[1], SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                
    

    elif(match_case == 'none'):
        #secondmatch_required = True
        #print('ya 1')

        NAME_TO_SEARCH = name_match[2]

        NAME_FOURTH_TRY = AssemblyHelpers.fourth_try_name_search(NAME_TO_SEARCH, natl_test, salary_boolean)

        MATCH_CASE_FOURTH_TRY = NAME_FOURTH_TRY[0]

        if(MATCH_CASE_FOURTH_TRY == 'single'): #YELLOW 1
            NAME_FOUND = NAME_FOURTH_TRY[1]
            money_thisplayer = AssemblyHelpers.find_money_info_from_name(NAME_FOUND, natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 

            money_found = money_thisplayer[0]
            search_db_return_case = money_thisplayer[1]
            online_lookup_required = money_thisplayer[2]

            if '.' in input_year_test:
                yearstr = input_year_test.split(".")[2]
                full_num = int('20' + yearstr)
            elif '-' in input_year_test:
                if len(input_year_test.split("-")[2]) == 4:
                    yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                    full_num = int(yearstr)
                elif len(input_year_test.split("-")[2]) == 2:
                    if len(input_year_test.split("-")[0]) == 4:
                        yearstr = input_year_test.split("-")[0]
                        full_num = int(yearstr)
                    else:
                        yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                        full_num = int('20' + yearstr)

            if ((money_found == 0) & (online_lookup_required == True)): #call the find_money_info_extended once you write it. inputs are = (NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, name_match[2])
                if ((leagues_value[leagues_value['Name'] == NAME_FOUND]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == NAME_FOUND]['Season'].max() == (full_num - 1))):
                    money_last_season = leagues_value[(leagues_value['Name'] == NAME_FOUND) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                    money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                else:
                    money_thisplayer = tryToFindPlayerMoney(search_db_return_case, NAME_FOUND, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                
            elif(search_db_return_case == 'working'):
                0==0
                #return to the end basically
            elif(search_db_return_case == 'working - imputed'):
                0==0
                #when ur done this should be removed. go into the function find_money_info and remove case

        elif(MATCH_CASE_FOURTH_TRY == 'multiple'):#RED 3

            print('here 4th try')
    
            LIST_TO_FILTER = NAME_FOURTH_TRY[1]
            money_thisplayer, name_results = filter_using_date_of_match(LIST_TO_FILTER, natl_test, input_year_test, salary_boolean) 
            if money_thisplayer == [0, 'Lookup Required', True, False]:
                SINGLE_NAME_MATCH_CORRECT_SEASON = name_results
                combination_key = (example_problem, full_num)
                combination_dict_matches[combination_key] = [SINGLE_NAME_MATCH_CORRECT_SEASON]
                print(f'added key {combination_key} to dict. value is {SINGLE_NAME_MATCH_CORRECT_SEASON}')
                player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value_large['Season'] == full_num)].reset_index()
                
                if len(player_bigvalue_dataset) >= 1:
                    #print(player_bigvalue_dataset.at[0, 'Market Value'])
                    #money_thisplayer[0] =
                    #print('working youre good')
                    money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                    if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                        #money_thisplayer[2] = True
                        money_thisplayer[1] = 'Large Dataset said 0'
                else:
                    if ((leagues_value[leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1))):
                        money_last_season = leagues_value[(leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                        money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                    else:
                        money_thisplayer = tryToFindPlayerMoney(money_thisplayer[1], SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
        
        elif(MATCH_CASE_FOURTH_TRY == 'none'): #SELENIUM
            #print('ya 2')
            original_db_name_for_search = name_match[2]

            accents_removed_name = remove_accents(original_db_name_for_search)
            #print(accents_removed_name)
            equal_names = [name for name in dataset_nationality if name == accents_removed_name]
            #(accents_removed_name in dataset_nationality)

            if equal_names:
                0==0
            else:
            #print
                matching_names = [name for name in dataset_nationality if name.endswith(accents_removed_name)]
                #print('weere here', matching_names)
                if len(matching_names) >= 2:
                    0==0 #more than 1 person remains
                elif len(matching_names) == 1:
                    0==0 #single matching name
                    #print('heylo')
                    money_thisplayer = AssemblyHelpers.find_money_info_from_name(matching_names[0], natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 
                else:
                    0==0 #no matches 
                    #print('nerr')
                    # MONEY_FOUND_ONLINE_0 = useSeleniumToFindMoney(original_db_name_for_search, natl_test, input_year_test, salary_boolean)
                    # money_thisplayer = [MONEY_FOUND_ONLINE_0, f'Online WAS Lookup Required - salary boolean is {salary_boolean}', True, False]
                    money_thisplayer = [0, 'Lookup Required', True, False]
    else:
        0==0 #RED 7

    return money_thisplayer, name_match


def remove_prefix(string):
    # Define pattern to match digits or '-' at the start of the string
    pattern = r'^[\d-]+'
    
    # Use re.sub to remove the matched pattern from the start of the string
    return re.sub(pattern, '', string)

In [3]:
combined_df = load_csv_dataset('combined_DF_terrikon_removed.csv')

failed_df = load_csv_dataset('failed_df_terrikon_removed.csv')

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_31625/899278855.py:55: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [5]:
len(combined_df)

57629

In [6]:
len(failed_df)

430

In [14]:
def add_correct_money(money_column):
    if money_column == 0:
        return 0

    if 'm' in money_column:
        value_rough = money_column.split('m')[0].split('€')[1]
        millinum = (float(value_rough) * 1000000)
        return millinum
    elif 'k' in money_column:
        #print(i, 'k')
        value_rough = money_column.split('k')[0].split('€')[1]
        millinum = (float(value_rough) * 1000)

        return millinum
    elif '-' in money_column:
        return 0
    else:
        return 0

def in_season_around(player_id, curr_season, age):
    if age == "Not Listed": age = 30
    age = int(age)
    year_before = int(curr_season) -1
    year_after = int(curr_season) + 1
    before_season = leagues_value[(leagues_value['PlayerID'] == player_id) & (leagues_value['Season'] == year_before)] #leagues_value_large #leagues_value
    after_season =  leagues_value[(leagues_value['PlayerID'] == player_id) & (leagues_value['Season'] == year_after)]
    if before_season.empty and after_season.empty:
        return -1, "Both Empty"
    if not before_season.empty and after_season.empty:
        if before_season.iloc[0]['Market Value'] != "-":
            if before_season.iloc[0]['Market Value'] == 0:
                return 0, "Only Before + Zero Before"
            else:
                if age >= 30:
                    return (float(add_correct_money(before_season.iloc[0]['Market Value'])) * .8), "Only Before" #add_correct_money
                else:
                    return (float(add_correct_money(before_season.iloc[0]['Market Value'])) * 1.2), "Only Before" #add_correct_money
        else:
            return 0, "Only Before X"
    if not after_season.empty and before_season.empty:
        
        if after_season.iloc[0]['Market Value'] != "-":
            if after_season.iloc[0]['Market Value'] == 0:
                return 0, "Only After + Zero After"
            else:
                if age <= 30:
                    return (float(add_correct_money(after_season.iloc[0]['Market Value'])) * .8), "Only After"
                else:
                    return (float(add_correct_money(after_season.iloc[0]['Market Value'])) * 1.2), "Only After"
        else:
            return 0, "Only After X"
    if not before_season.empty and not after_season.empty:
        #print('hi')
        before_season_val = before_season.iloc[0]['Market Value']
        after_season_val = after_season.iloc[0]['Market Value']
        #print(before_season_val, after_season_val)
        if before_season_val != "-" and after_season_val != "-":
            # if type(before_season_val) == str:
            #     before_season_val = add_correct_money(before_season_val)
            # if type(after_season_val) == str:
            #     after_season_val = add_correct_money(after_season_val)
            return ((float(add_correct_money(before_season.iloc[0]['Market Value'])) + float(add_correct_money(after_season.iloc[0]['Market Value'])) / 2)), "Used Average of Season Before and After"
        elif before_season_val != "-" and after_season_val == "-":
            if age >= 30:
                return (float(add_correct_money(before_season.iloc[0]['Market Value']) * .8)), "Used Season Before * .8 But Had Both"
            else:
                return (float(add_correct_money(before_season.iloc[0]['Market Value']) * 1.2)), "Used Season Before * 1.2 But Had Both"
        elif before_season_val == "-" and after_season_val != "-":
            if age <= 30:
                return (float(add_correct_money(after_season.iloc[0]['Market Value']) * .8)), "Used Season After * .8 But Had Both"
            else:
                return (float(add_correct_money(after_season.iloc[0]['Market Value']) * 1.2)), "Used Season After * 1.2 But Had Both"
        else:
            return 0, "Had Both but Both Were Empty"


def find_in_transfermarkt(player_name, season, country_code, database_name):
    #print(player_name, country_code, season)
    row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == season) & (database_name['Team 1 Code'] == country_code)]
    one_up_row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == int(season)+1) & (database_name['Team 1 Code'] == country_code)]
    one_down_row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == int(season)-1) & (database_name['Team 1 Code'] == country_code)]
    print(len(row), len(one_down_row), len(one_up_row))
    final_value = 0
    if not row.empty:
        #print('yeeea')
        player_id = row.iloc[0]['PlayerID']
        age = row.iloc[0]['Age']
        marketval = row.iloc[0]['Market Value']
        if type(marketval) == str:
            #print('der')
            if marketval == '-':
                marketval = 0
            elif '€' in marketval:
                marketval = add_correct_money(marketval)
            marketval = float((marketval)) #add_correct_money
        if marketval != 0:
            #print('wooo')
            #print('***', player_name, marketval)
            return marketval, "Found in Curr Season"
        else:
            #print(marketval)
            new_mv, reason = in_season_around(player_id, season, age)
            print(player_name, new_mv, reason)
            if new_mv == -1 or new_mv == 0 or new_mv == "-":
                return 0, reason
            else:
                return new_mv, reason
    elif not one_up_row.empty:
        player_id = one_up_row.iloc[0]['PlayerID']
        age = one_up_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    elif not one_down_row.empty:
        player_id = one_down_row.iloc[0]['PlayerID']
        age = one_down_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    else:
        return 0, "Did Not Find in Curr Season, One Up, or One Down"
    

### Euros qualifiers

In [84]:
lineups_euros_qualifiers = load_csv_dataset('Temp Lineup Data - Euros Qualifiers Lineups.csv')
#fix country names in the TeamName and Match 

# for index, row in lineups_euros_qualifiers.iterrows():
#     if row['TeamName'] == 'Czechia':
#         lineups_euros_qualifiers.at[index, 'TeamName'] = 'Czech'
#         lineups_euros_qualifiers.at[index, 'Match'] = lineups_euros_qualifiers.at[index, 'Match'].replace('Czechia', 'Czech')
#     if row['TeamName'] == 'Türkiye':
#         lineups_euros_qualifiers.at[index, 'TeamName'] = 'Turkey'
#         lineups_euros_qualifiers.at[index, 'Match'] = lineups_euros_qualifiers.at[index, 'Match'].replace('Türkiye', 'Turkey')
#     if row['TeamName'] == 'Bosnia':
#         lineups_euros_qualifiers.at[index, 'TeamName'] = 'Bosnia-Herzegovina'
#         lineups_euros_qualifiers.at[index, 'Match'] = lineups_euros_qualifiers.at[index, 'Match'].replace('Bosnia', 'Bosnia-Herzegovina')

#     if 'Czechia' in row['Match']:
#         lineups_euros_qualifiers.at[index, 'Match'] = lineups_euros_qualifiers.at[index, 'Match'].replace('Czechia', 'Czech')
#     if 'Türkiye' in row['Match']:
#         lineups_euros_qualifiers.at[index, 'Match'] = lineups_euros_qualifiers.at[index, 'Match'].replace('Türkiye', 'Turkey')
#     if 'Bosnia' in row['Match']:
#         if 'Bosnia-Herzegovina' in row['Match']:
#             0==0
#         else:
#             lineups_euros_qualifiers.at[index, 'Match'] = lineups_euros_qualifiers.at[index, 'Match'].replace('Bosnia', 'Bosnia-Herzegovina')

In [70]:
lineups_euros_qualifiers['Code'] = lineups_euros_qualifiers['TeamName'].apply(get_country_code)


In [89]:
#lineups_data[lineups_data["TeamName"] == 'Kazakhstan']
lineups_data[lineups_data['Match'] == 'Wales vs Belgium']

,TeamName,Starter_1,Starter_2,Starter_3,Starter_4,Starter_5,Starter_6,Starter_7,Starter_8,Starter_9,...,Unused_12,Unused_13,Unused_14,Unused_15,Competition,Date,Match,Code,Found?,Match ID
790,Wales,-,-,-,-,-,-,-,-,-,...,-,-,-,-,"Euros 2016, Qualifiers",2015-12-06,Wales vs Belgium,WA,False,400
791,Belgium,-,-,-,-,-,-,-,-,-,...,-,-,-,-,"Euros 2016, Qualifiers",2015-12-06,Wales vs Belgium,BE,False,400
2206,Wales,1. Hennessey,2. Gunter,3. Taylor N.,4. Davies,5. Chester,6. A. Williams,7. Allen,9. Robson-Kanu Hal,10. Ramsey,...,-,-,-,-,Euros 2016,2016-01-07,Wales vs Belgium,WA,False,1112
2207,Belgium,1. Courtois,2. Alderweireld,15. Denayer,16. Meunier,21. Lukaku J.,4. Nainggolan,6. Witsel,7. De Bruyne,11. Carrasco,...,-,-,-,-,Euros 2016,2016-01-07,Wales vs Belgium,BE,False,1112
3460,Wales,1. Hennessey,3. Williams N.,4. Davies,5. Mepham,6. Rodon,14. Roberts C.,7. Allen,15. Ampadu,20. James D.,...,-,-,-,-,Nations League 2022-2023,2022-11-06,Wales vs Belgium,WA,False,1749
3461,Belgium,13. Casteels,3. Theate,4. Boyata,15. Meunier,19. Dendoncker,6. Witsel,7. De Bruyne,8. Tielemans,11. Carrasco,...,-,-,-,-,Nations League 2022-2023,2022-11-06,Wales vs Belgium,BE,False,1749
6828,Wales,12 D. Ward,5 C. Mepham,6 J. Rodon,4 B. Davies,14 C. Roberts,7 J. Allen,16 J. Morrell,3 N. Williams,10 A. Ramsey,...,-,-,-,-,"World Cup Qualifiers, Europe, 2022",2021-11-16,Wales vs Belgium,WA,False,3448
6829,Belgium,13 K. Casteels,21 T. Castagne,4 D. Boyata,2 A. Theate,15 T. Meunier,6 A. Witsel,8 H. Vanaken,10 T. Hazard,7 K. De Bruyne,...,-,-,-,-,"World Cup Qualifiers, Europe, 2022",2021-11-16,Wales vs Belgium,BE,False,3448


In [88]:
counter = 0
for index, row in lineups_euros_qualifiers.iterrows():
    date = row['Date']
    #date = convert_date(row['Date'])
    #date_range = pd.date_range(date - pd.Timedelta(days=1), date + pd.Timedelta(days=1))
    matches = lineups_data[(lineups_data['Date'] == date) & (lineups_data['Match'] == row['Match'])]
    if not matches.empty:
        0==0
        #lineups_euros_qualifiers.at[index, 'Match ID'] = matches.iloc[0]['Match ID']
    else:
        print(index, row['Match'], row['Date'])
        counter += 1
counter

30 Kazakhstan vs Turkey Jun 12, 2015
31 Kazakhstan vs Turkey Jun 12, 2015
32 Latvia vs Netherlands Jun 12, 2015
33 Latvia vs Netherlands Jun 12, 2015
34 Iceland vs Czech Jun 12, 2015
35 Iceland vs Czech Jun 12, 2015
36 Czech vs Kazakhstan Sep 3, 2015
37 Czech vs Kazakhstan Sep 3, 2015
38 Netherlands vs Iceland Sep 3, 2015
39 Netherlands vs Iceland Sep 3, 2015
40 Turkey vs Latvia Sep 3, 2015
41 Turkey vs Latvia Sep 3, 2015
42 Latvia vs Czech Sep 6, 2015
43 Latvia vs Czech Sep 6, 2015
44 Turkey vs Netherlands Sep 6, 2015
45 Turkey vs Netherlands Sep 6, 2015
46 Iceland vs Kazakhstan Sep 6, 2015
47 Iceland vs Kazakhstan Sep 6, 2015
62 Bosnia-Herzegovina-Herzegovina vs Cyprus Sep 9, 2014
71 Wales vs Bosnia-Herzegovina-Herzegovina Oct 10, 2014
74 Bosnia-Herzegovina-Herzegovina vs Belgium Oct 13, 2014
83 Israel vs Bosnia-Herzegovina-Herzegovina Nov 16, 2014
87 Andorra vs Bosnia-Herzegovina-Herzegovina Mar 28, 2015
90 Wales vs Belgium Jun 12, 2015
91 Wales vs Belgium Jun 12, 2015
92 Andorra vs

658

In [56]:
import pandas as pd
import datetime as datetime

def convert_date(input_date):
    formats = ["%d.%m.%y", "%d-%m-%Y", "%d-%b-%y", "%Y-%m-%d", "%b %d, %Y"]
    target_format = "%Y-%m-%d"

    for date_format in formats:
        try:
            # Try parsing the input date using the current format
            parsed_date = datetime.strptime(input_date, date_format)
            # If successful, convert it to the target format
            result_date = parsed_date.strftime(target_format)
            #year = parsed_date.year
            #return year
            return result_date
        except ValueError:
            #print(input_date)
            pass


#old_lineups_data['Formatted Date'] = old_lineups_data['Date'].apply(convert_date)
lineups_data['Date'] = pd.to_datetime(lineups_data['Date'])

dates_orig = lineups_euros_qualifiers['Date']
lineups_euros_qualifiers['Date'] = pd.to_datetime(lineups_euros_qualifiers['Date'])

lineups_euros_qualifiers['Match ID'] = ""
counter = 0
for index, row in lineups_euros_qualifiers.iterrows():
    date = row['Date']
    #date = convert_date(row['Date'])
    date_range = pd.date_range(date - pd.Timedelta(days=1), date + pd.Timedelta(days=1))
    matches = lineups_data[(lineups_data['Date'].isin(date_range)) & (lineups_data['Code'] == row['Code'])]
    if not matches.empty:
        lineups_euros_qualifiers.at[index, 'Match ID'] = matches.iloc[0]['Match ID']
    else:
        print(index, row['Match'])
        counter += 1
print(counter)
lineups_euros_qualifiers['Date'] = dates_orig

30 Kazakhstan vs Türkiye
31 Kazakhstan vs Turkey
32 Latvia vs Netherlands
33 Latvia vs Netherlands
34 Iceland vs Czechia
35 Iceland vs Czech Republic
36 Czech Republic vs Kazakhstan
37 Czechia vs Kazakhstan
38 Netherlands vs Iceland
39 Netherlands vs Iceland
40 Turkey vs Latvia
41 Türkiye vs Latvia
42 Latvia vs Czechia
43 Latvia vs Czech Republic
44 Turkey vs Netherlands
45 Türkiye vs Netherlands
46 Iceland vs Kazakhstan
47 Iceland vs Kazakhstan
90 Wales vs Belgium
91 Wales vs Belgium
92 Andorra vs Cyprus
93 Andorra vs Cyprus
94 Bosnia vs Israel
95 Bosnia vs Israel
96 Belgium vs Bosnia
97 Belgium vs Bosnia
98 Cyprus vs Wales
99 Cyprus vs Wales
100 Israel vs Andorra
101 Israel vs Andorra
102 Wales vs Israel
103 Wales vs Israel
104 Bosnia vs Andorra
105 Bosnia vs Andorra
106 Cyprus vs Belgium
107 Cyprus vs Belgium
120 Spain vs Macedonia
121 Spain vs Macedonia
122 Ukraine vs Slovakia
123 Ukraine vs Slovakia
124 Luxembourg vs Belarus
125 Luxembourg vs Belarus
126 Slovakia vs Spain
127 Slov

In [12]:
lineups_euros_qualifiers = load_csv_dataset('Temp Lineup Data - Euros Qualifiers Lineups.csv')

In [13]:

for i in range(0, len(lineups_euros_qualifiers)):
    if lineups_euros_qualifiers.at[i, 'Date'].startswith('Sat '):
        lineups_euros_qualifiers.at[i, 'Date'] = lineups_euros_qualifiers.at[i, 'Date'].split('Sat ')[1]
    elif lineups_euros_qualifiers.at[i, 'Date'].startswith('Sun '):
        lineups_euros_qualifiers.at[i, 'Date'] = lineups_euros_qualifiers.at[i, 'Date'].split('Sun ')[1]
    elif lineups_euros_qualifiers.at[i, 'Date'].startswith('Wed '):
        lineups_euros_qualifiers.at[i, 'Date'] = lineups_euros_qualifiers.at[i, 'Date'].split('Wed ')[1]
    elif lineups_euros_qualifiers.at[i, 'Date'].startswith('Tue '):
        lineups_euros_qualifiers.at[i, 'Date'] = lineups_euros_qualifiers.at[i, 'Date'].split('Tue ')[1]
    elif lineups_euros_qualifiers.at[i, 'Date'].startswith('Thu '):
        lineups_euros_qualifiers.at[i, 'Date'] = lineups_euros_qualifiers.at[i, 'Date'].split('Thu ')[1]
    elif lineups_euros_qualifiers.at[i, 'Date'].startswith('Mon '):
        lineups_euros_qualifiers.at[i, 'Date'] = lineups_euros_qualifiers.at[i, 'Date'].split('Mon ')[1]
    elif lineups_euros_qualifiers.at[i, 'Date'].startswith('Fri '):
        lineups_euros_qualifiers.at[i, 'Date'] = lineups_euros_qualifiers.at[i, 'Date'].split('Fri ')[1]
lineups_euros_qualifiers['Date'] = pd.to_datetime(lineups_euros_qualifiers['Date'])

In [15]:
lineups_euros_qualifiers['Status New'] = ''

In [16]:
lookup_dataset_1 = pd.DataFrame()


not_found_dataset = pd.DataFrame()


In [21]:
for index, row in lineups_euros_qualifiers.iterrows():
    match_name = row['Match']
    date = row['Date']
    competition = row['Competition']
    lineups_data_this_game = lineups_data[(lineups_data['Match'] == match_name) & (lineups_data['Date'] == date) & (lineups_data['Competition'] == competition)]
    if len(lineups_data_this_game) == 0:
        print(index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [19]:
# add country codes using the countriescodes

# add match IDs 
    #for each row look up that match date and competition 
    #check if it's in the big lineups DF. 
        #if it is not print 
        #if it is add the match ID 

In [17]:
counter = 0
salary_boolean = False
for k in range(0, len(lineups_euros_qualifiers)): #lineups_data_concacaf_conmebol

    nationality_this = lineups_euros_qualifiers.at[k, 'TeamName']
    competition_this = lineups_euros_qualifiers.at[k, 'Competition']
    #nationality_this = lineups_data.at[k, 'Match']
    date_this = lineups_euros_qualifiers.at[k, 'Date']

    
    country_code_this = lineups_euros_qualifiers.at[k, 'Code']
    date_this = str(date_this)
    if ' 00:00:00' in date_this:
        date_this = date_this.split(' 00:00:00')[0]
    print(k, nationality_this, date_this)
    for i in range(2, 13): 
        number = -999
        reason = ''

        column = lineups_euros_qualifiers.columns[i]
        player_this = remove_prefix(lineups_euros_qualifiers.at[k, f'{column}'])
        if player_this[0] == ' ':
            player_this = player_this[1:]

        if player_this == '-':
            0==0
        else:
            #print(player_this)
            
            if '-' in date_this:
                if len(date_this.split("-")[2]) == 4:
                    yearstr = date_this.split("-")[2] #this is a 4 digit number
                    full_num = int(yearstr)
                elif len(date_this.split("-")[2]) == 2:
                    if len(date_this.split("-")[0]) == 4:
                        yearstr = date_this.split("-")[0]
                        full_num = int(yearstr)
                    else:
                        yearstr = date_this.split("-")[2] #this is a 2 digit number
                        full_num = int('20' + yearstr)
        dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                                        (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
        if player_this in dataset_nationality:
            0==0
            
            number, reason = find_in_transfermarkt(player_this, full_num, country_code_this, leagues_value)
            #print(player_this, reason, add_correct_money(number))

            #add a new row to lookup dataset 1 
            if type(number) == float:
                0==0
            else:
                number = add_correct_money(number)
            
            lineups_euros_qualifiers.at[k, 'Status New'] = 'Done'

            data_row = {
                'Match ID': lineups_euros_qualifiers.at[k, 'Match ID'],
                'Competition': competition_this,
                'Match': lineups_euros_qualifiers.at[k, 'Match'],
                'Date': date_this,
                'Nationality': nationality_this,
                'Team Country Code': country_code_this,
                'Season': full_num,
                'Status': 'SUCCESS',
                'Name': player_this, #this will be zero if a fail happens 
                'Market Value': number,
                'Lookup Still Required?': False,
                'Lookup Return Case': reason,
                'Impute Required?': False,
                'Name(s) Found': player_this,
                'ORIGINAL JERSEY': lineups_euros_qualifiers.at[k, f'{column}'],
                'Match Case': 'single' #single, multiple, none or cyrillic
            }
            lookup_dataset_1 = pd.concat([lookup_dataset_1, pd.DataFrame([data_row])], ignore_index=True)
            
        else:
            print(player_this, 'nope')
            # counter += 1
            dataset_nationality_all_years = leagues_value[(leagues_value['Team 1 Code'] == country_code_this)]['Name'].unique()
            if player_this in dataset_nationality_all_years:
                
                print('***Found Him', player_this)
                data_row = {
                    'Match ID': lineups_euros_qualifiers.at[k, 'Match ID'],
                    'Competition': competition_this,
                    'Match': lineups_euros_qualifiers.at[k, 'Match'],
                    'Date': date_this,
                    'Nationality': nationality_this,
                    'Team Country Code': country_code_this,
                    'Season': full_num,
                    'Status': 'FAIL',
                    'Name': player_this, #this will be zero if a fail happens 
                    'Market Value': 0,
                    'Lookup Still Required?': True,
                    'Lookup Return Case': 'Did Not Find in Curr Season, One Up, or One Down', 
                    'Impute Required?': False,
                    'Name(s) Found': player_this,
                    'ORIGINAL JERSEY': lineups_euros_qualifiers.at[k, f'{column}'],
                    'Match Case': 'single' #single, multiple, none or cyrillic
                }        
                not_found_dataset = pd.concat([not_found_dataset, pd.DataFrame([data_row])], ignore_index=True)    
            else:
                print('DID NOT FIND ', player_this)
                counter += 1
                data_row = {
                    'Match ID': lineups_euros_qualifiers.at[k, 'Match ID'],
                    'Competition': competition_this,
                    'Match': lineups_euros_qualifiers.at[k, 'Match'],
                    'Date': date_this,
                    'Nationality': nationality_this,
                    'Team Country Code': country_code_this,
                    'Season': full_num,
                    'Status': 'FAIL',
                    'Name': player_this, #this will be zero if a fail happens 
                    'Market Value': 0,
                    'Lookup Still Required?': True,
                    'Lookup Return Case': 'Did Not Find in any Season', #Did Not Find in Curr Season, One Up, or One Down
                    'Impute Required?': False,
                    'Name(s) Found': player_this,
                    'ORIGINAL JERSEY': lineups_euros_qualifiers.at[k, f'{column}'],
                    'Match Case': 'single' #single, multiple, none or cyrillic
                }        
                not_found_dataset = pd.concat([not_found_dataset, pd.DataFrame([data_row])], ignore_index=True)     
            
    #dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == country_code_this]['Name'].unique()
        #make the dataset nationality equal to only the season before , current or after 

print(counter)



KeyError: 'Code'